# This project will conduct some basic analysis on vehicles in the US.

### First we will upload some of the libraries that we need to conduct our analysis.
We will need to add them to our interperter, I will be using anaconda and it's base version since it has most of the packages we need installed already. We will need to install altair, streamlit, plotly, seaborn. If you want we can also do matplotlib as an alternative for our plots. If you need an enviornment to use I created one name proj_env and it is in the files associated with the project under environment.yml you can see what is installed with it. 

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import streamlit as st
import plotly.express as px



Next we are going to read the csv file we want to investigate. In this case vehicles_us.csv, then print the first couple of lines.

In [24]:
df= pd.read_csv('/Users/12629/tripleten_project/tripleten_project/vehicles_us.csv')
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


Firstly, we can see that the data is being loaded in correctly and we do not need to adjust reading the files with any arguements. Next we need to find how many missing or duplicate values we have. We will do this with .info() and .describe().

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [26]:
df.describe()

,price,model_year,cylinders,odometer,is_4wd,days_listed
count,51525.000000,47906.000000,46265.000000,43633.000000,25572.0,51525.00000
mean,12132.464920,2009.750470,6.125235,115553.461738,1.0,39.55476
std,10040.803015,6.282065,1.660360,65094.611341,0.0,28.20427
min,1.000000,1908.000000,3.000000,0.000000,1.0,0.00000
25%,5000.000000,2006.000000,4.000000,70000.000000,1.0,19.00000
50%,9000.000000,2011.000000,6.000000,113000.000000,1.0,33.00000
75%,16839.000000,2014.000000,8.000000,155000.000000,1.0,53.00000
max,375000.000000,2019.000000,12.000000,990000.000000,1.0,271.00000


Looks like we have some some missing values in the model_year, cylinders, odometer, paint_color, and is_4wd. Lets address those columns.

In [27]:
df['model_year'].isnull().sum()

3619

In [28]:
df['model_year'].value_counts(dropna=False)

model_year
NaN       3619
2013.0    3549
2012.0    3468
2014.0    3448
2011.0    3375
          ... 
1948.0       1
1961.0       1
1936.0       1
1949.0       1
1929.0       1
Name: count, Length: 69, dtype: int64

Okay we want to deal with the missing values. Since the dtypes for model_year is float we should try to keep that consistant so we will change our NaN values with the median value so that it is less likely to influence our data if we run any statisics on it. 

In [30]:
df['model_year'] = df['model_year'].fillna(df['model_year'].median())

df['model_year'].isnull().sum()

0

Next we will work on cylinders. Again we will make the NaN values the median so that the statistics will remain more accurate.

In [31]:
df['cylinders'].value_counts(dropna=False)

cylinders
8.0     15844
6.0     15700
4.0     13864
NaN      5260
10.0      549
5.0       272
3.0        34
12.0        2
Name: count, dtype: int64

In [32]:
df['cylinders']= df['cylinders'].fillna(df['cylinders'].median())
df['cylinders'].isnull().sum()

0

We will do the odometer next. Same thing, we will use the median. This will make it so we can see the skew in the same fashion. If we need to run stats on it we can subtract the unknown values from the median first before processing it. 

In [33]:
df['odometer']= df['odometer'].fillna(df['odometer'].median())
df['odometer'].isnull().sum()

0

Next we will do paint_color. This time we will do unknown for the replacement as it will be it's own category in the object dtype.

In [34]:
df['paint_color']= df['paint_color'].fillna('unknown')
df['paint_color'].isnull().sum()

0

For is_4w we will make the NaN a 0 so that it is easy to see that the car is either 1= 4w or 0= not 4w or is unknown.

In [35]:
df['is_4wd']= df['is_4wd'].fillna(0)
df['is_4wd'].isnull().sum()

0

Now we will check .info() to see if everything is cleared up and still has the same dtype.

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    51525 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     51525 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      51525 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   51525 non-null  object 
 10  is_4wd        51525 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


Now that the data is set for further anaylisis in terms of filled data we need to check for any duplicates.

In [37]:
df.duplicated().sum()

0

Well that makes it easy for us. Now we can move on to the next part.